# Dataset Transformation

**Bagian di bawah ini disalin dari file ```10_DatasetDataloader.ipynb```**

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

Pada bagian ini kita melakukan sedikit modifikasi:
- Pada bagian ```def __init__```, kita menambahkan ```self.transform = transform```, juga tidak lupa pada bagian argumen ```transform```
- Pada bagian ```def __getitem__```, kita menambahkan ```self.transform```

In [ ]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        xy = np.loadtxt('wine_dataset/wine.csv', delimiter=',', dtype=np.float32,
                        skiprows=1)  # skip row 1 karena row 1 adalah header
        
        self.n_samples = len(xy)
        
        self.x = xy[:, 1:]  # mengabaikan kolom indeks-0 karena kolom tersebut adalah kelasnya
        self.y = xy[:, [0]]
        
        self.transform = transform

    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

    def __len__(self):
        return self.n_samples

## Membuat ```class ToTensor()```
- Kelas ini berfungsi untuk mengkonversikan nilai masukan dan target ke dalam bentuk tensor.
- ```__call__``` adalah method bawaan dari python yang memungkinkan pengguna membuat kelas yang instancenya dapat dipanggil sebagai fungsi.
- Lihat lebih lanjut penjelasannya pada [tautan berikut ini](https://www.geeksforgeeks.org/__call__-in-python/)

In [ ]:
class ToTensor():
    def __call__(self, sample):
        x, y = sample
        return torch.from_numpy(x), torch.from_numpy(y)

dataset = WineDataset(transform=None)

Mencoba mengambil sampel data awal. Dan pada bagian ini, tidak ada transformasi yang dilakukan

In [ ]:
first_data = dataset[0]
contoh_x, contoh_y = first_data
print(contoh_x, contoh_y)
print(contoh_x.shape, contoh_y.shape)
print(type(contoh_x), type(contoh_y))

## Membuat ```class MulTransform()```
- Kelas ini berfungsi untuk mengalikan input dengan faktor pengali
- Nilai kembalian dari fungsi ini adalah input (x) yang telah dikali dan target (y) yang tetap.

In [ ]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, sample):
        x, y = sample
        x = x * self.factor
        return x, y
        

## Membuat Beberapa Transformasi Menjadi Satu
- Dapat diimplementasikan dengan ```torchvision.transforms.Compose```
- Tujuannya adalah, kita menggabungkan transformasi yang kita buat pada kelas ```ToTensor``` dan ```MulTransform``` menjadi sebuah fungsi transformasi

In [ ]:
import torchvision
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])

dataset = WineDataset(transform=composed)

first_data = dataset[0]
contoh_x, contoh_y = first_data
print(contoh_x, contoh_y)
print(contoh_x.shape, contoh_y.shape)
print(type(contoh_x), type(contoh_y))

Dapat kita amati beberapa hal:
- Setelah menggunakan fungsi transformasi ```composed```, kita akan mendapatkan input dan target yang telah dikonversikan ke dalam bentuk tensor.
- Nilai dari ```contoh_x``` dan ```contoh_y``` setelah ditransformasi juga berbeda dengan sebelum di transformasikan.
- ```torchvision.transforms.Compose``` memungkinkan kita untuk menggabungkan beberapa fungsi transformasi.